In [ ]:
%reset 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from matplotlib import cm
import pylab
from scipy.optimize import fsolve
import matplotlib.patches as patches

DataPath_rate='./Data/Data_F8_Rate_PILOCm_intervKsb'

%run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
%run ./DataGenerationScripts/set_size.ipynb # loading function for pdf output sizer
%run ./DataGenerationScripts/aux_state_space.ipynb # loading auxiliarry functions for state space vizualization
%run ./DataGenerationScripts/load_animal_model_data.ipynb # loading function for pdf output sizer
%run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading function for Seizure Rate calculation

In [ ]:
##### This cell checks whether data exists. If not, it is being generated #####
if not os.path.isdir(DataPath_rate):
    os.makedirs(DataPath_rate)
if not os.path.isfile(DataPath_rate+'/Sim_1.npy'):
    print('Data not found. Running Data Generation script')
    RunningFromFigureScript=True
    %run ./DataGenerationScripts/DataGeneration_F8_Rate_PILOCm_intervKsb.ipynb

In [ ]:
t_interv_windows=np.array([[-0.1,-0.01],# no intervention
                            [0,365*10], # permanent (until t_end)
                            [0,10*7], # 10 weeks
                            [0,2*7], # 2 weeks
                            [0,5*7], # 5 weeks
                            [2*7,7*7], # 5 weeks with 2 weeks delay
                            [5*7,10*7]]) # 5 weeks with 5 weeks delay 
leg=['REF','I','II','III','IV','V','VI']
t_interv_windows=t_interv_windows/7 # express everything in weeks

In [ ]:
font = {'size'   : 8}
# plt.rc('font', family='sans-serif') 
# plt.rc('font', serif='Helvetica Neue')
plt.rc('font',**{'family':'sans-serif','serif':['Helvetica Neue']})
plt.rc('font', **font)
plt.rc('text', usetex=True)

viridis = cm.get_cmap('viridis', 12)
color_scheme_7examples = ['r','b',viridis(0.0)[0:3],viridis(0.3)[0:3],viridis(0.55)[0:3],viridis(0.75)[0:3],viridis(0.95)[0:3]]
alpha_value=0.6
size_scaler = 1
marker_sizer=2.5
lw_7=[0.25/size_scaler,0.25/size_scaler,1/size_scaler,1/size_scaler,1/size_scaler,1/size_scaler,1/size_scaler]
alph_7=[1,1,alpha_value,alpha_value,alpha_value,alpha_value,alpha_value]

h=2.8*0.393701 #[cm] to  modification [inch] Height of output figure
w=2.8*0.393701 #[cm] to  modification [inch] Width of output figure
margin=0.4 # [1] margin ratio of output figure layout
size_scaler=2.5

## Figure 8a  

In [ ]:
t_end=12 #weeks
listt=np.load(DataPath_rate+'/Sim_'+str(1)+'.npy',allow_pickle=True)  
params=listt[0]

fig = matplotlib.pyplot.gcf()
ax = fig.add_subplot(1, 1, 1)

for ii in range (1,len(t_interv_windows[:,0])+1):
    rect = patches.Rectangle((t_interv_windows[ii-1,0],ii-0.875),t_interv_windows[ii-1,1]-t_interv_windows[ii-1,0],0.75,linewidth=1,edgecolor='none',facecolor=color_scheme_7examples[ii-1],alpha=alpha_value)
    ax.add_patch(rect)
 

rect = patches.Rectangle((0,0),params['IBDR_E_duration'][1]/7,10,linewidth=1,edgecolor='none',facecolor='k',alpha=0.3)
ax.add_patch(rect)


plt.xlim([0,t_end])
plt.xticks(np.append(t_interv_windows[2:,0], t_interv_windows[2:,1]))
plt.ylim([0,ii+0.25])
plt.yticks(np.linspace(0.5,-0.5+len(t_interv_windows[:,1]),len(t_interv_windows[:,1])),
           ['REF','I','II','III','IV','V','VI'])


plt.title(r'Pilocarpine intervention with $K_{S\rightarrow B}$ modulation')
plt.xlabel('weeks after injury onset')
plt.ylabel('Intervention time windows')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
plt.gcf().subplots_adjust(top=1-margin/2)
plt.gcf().subplots_adjust(right=1-margin/2)
set_size(2*w,w) 
fig.savefig('Figures/Fig8/Fig_8a.pdf')

## Figure 8b

In [ ]:
fig = matplotlib.pyplot.gcf()
ax = fig.add_subplot(1, 1, 1)

vis_order=[0,3,4,5,6,2,1] # order to make sure that all lines are seen

for ii in vis_order:
# for ii in range (0,len(t_interv_windows[:,0])):
    listt=np.load(DataPath_rate+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)  
    t_vec=listt[1]['t_vec']
    D_vec=listt[1]['D_vec']
    plt.plot(t_vec, D_vec,color=color_scheme_7examples[ii],linewidth=lw_7[ii],label=leg[ii],alpha=alph_7[ii])

ax.legend(bbox_to_anchor=(-1, 1), loc='upper left')

rect = patches.Rectangle((0,-1),params['IBDR_E_duration'][1],2,linewidth=1,edgecolor='none',facecolor='k',alpha=0.2)
ax.add_patch(rect)

plt.title(r'Pilocarpine intervention with $K_{S\rightarrow B}$ modulation')
plt.xlim([0,7])
plt.ylim([0,0.5])
ax.set_ylabel(r'Extent of neuronal loss $D(t)$')

plt.xlabel('days after injury')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
plt.gcf().subplots_adjust(top=1-margin/2)
plt.gcf().subplots_adjust(right=1-margin/2)
set_size(2*w,w) 
fig.savefig('Figures/Fig8/Fig_8b.pdf')

## Figure 8c

In [ ]:
lambda_max=load_default_parameters('stochastic')['lambda_max']

fig = matplotlib.pyplot.gcf()
ax = fig.add_subplot(1, 1, 1)

vis_order=[0,3,4,5,6,2,1] # order to make sure that all lines are seen

for ii in vis_order:
# for ii in range (0,len(t_interv_windows[:,0])):
    listt=np.load(DataPath_rate+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)  
    params=listt[0]
    params['lambda_max']=lambda_max
    t_vec=listt[1]['t_vec']/365 # express time in years after injury 
    I_vec=listt[1]['I_vec']
    R_vec=listt[1]['R_vec']
    plt.plot(t_vec, SeizureRate(I_vec,R_vec,params),color=color_scheme_7examples[ii],linewidth=lw_7[ii],label=leg[ii],alpha=alph_7[ii])

ax.legend(bbox_to_anchor=(-1, 1), loc='upper left')

plt.title(r'Pilocarpine intervention with $K_{S\rightarrow B}$ modulation')
# plt.xlim([0,365*10])
plt.ylim([0,15])
ax.set_ylabel(r'Seizure rate, $\frac{\textnormal{seizures}}{\textnormal{day}}$')

plt.xlabel('years after injury')


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
plt.gcf().subplots_adjust(top=1-margin/2)
plt.gcf().subplots_adjust(right=1-margin/2)
set_size(2*w,w) 
fig.savefig('Figures/Fig8/Fig_8c.pdf')

## Figure 8d

In [ ]:
params=load_default_parameters('rate')
locals().update(params)
    
D_const=0.2

B_max = 1.1
R_max = 1.1
B_min = -0.1
R_min = -0.1

R,B = np.mgrid[R_min:R_max:5000j, B_min:B_max:5000j]

FP=[];
f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
U = 1./tau_B*(-B+k_IB*k_BI*B+f)
V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

#     velocity = np.sqrt(U*U + V*V)                
FP=fixedpointsfinder(fun_dbdt,-0.1,1,100000) # Here we will check for steady states on given interval with given discritization step


pylab.ion()
fig0= plt.figure()
ax0 = fig0.add_subplot(1, 1, 1)
strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.5,1.5])

for cc in range(len(FP)):
    if FP[cc][0]>=Theta:
        if D_const == D_m:
            col='black' # globally stable since here dD_dt = 0 , despite B>=Theta
        else:
            col='grey' # stable only with D holded at const
    else:
        col='black' # globally stabe since dD_dt = 0 due to B<Theta

    mrksize = 10
    if FP[cc][1]=='Unstable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
    if FP[cc][1]=='Semistable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'x',color=col, markersize=mrksize/marker_sizer-3/marker_sizer)
    if FP[cc][1]=='Stable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)

threshold_position = Theta/k_BI
plt.plot([threshold_position, threshold_position],[R_min,R_max],'--',color='tab:red',linewidth = 3.0/marker_sizer)


plt.xlabel("Extent of blood-brain \n" 
           r"barrier disruption $B \approx I$")
plt.ylabel("Degree of circuit \n"
           r"remodeling $R$")
plt.title('$D$='+str(np.round(D_const*100)/100),fontsize=font['size'])

ax0.set_xlim([B_min,B_max])
ax0.set_ylim([R_min,R_max])

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
set_size(1.0*w,1.0*w)
fig0.savefig('Figures/Fig8/Fig_8d1.pdf')  

In [ ]:
params=load_default_parameters('rate')
params['K_SB']=params['K_SB']*0.01
locals().update(params)
    
D_const=0.2

B_max = 1.1
R_max = 1.1
B_min = -0.1
R_min = -0.1

R,B = np.mgrid[R_min:R_max:5000j, B_min:B_max:5000j]

FP=[];
f = K_SB*(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)-1)/(np.exp(k_IS*np.square(k_BI*B)+k_RS*R)+1)
U = 1./tau_B*(-B+k_IB*k_BI*B+f)
V = 1./tau_R*(-R+k_BR*B+k_DR*D_const) 

#     velocity = np.sqrt(U*U + V*V)                
FP=fixedpointsfinder(fun_dbdt,-0.1,1,100000) # Here we will check for steady states on given interval with given discritization step


pylab.ion()
fig0= plt.figure()
ax0 = fig0.add_subplot(1, 1, 1)
strm = ax0.streamplot(B, R, U, V, linewidth=1/marker_sizer, color=[0.75,0.75,0.75], density=[0.5,1.5])

for cc in range(len(FP)):
    if FP[cc][0]>=Theta:
        if D_const == D_m:
            col='black' # globally stable since here dD_dt = 0 , despite B>=Theta
        else:
            col='grey' # stable only with D holded at const
    else:
        col='black' # globally stabe since dD_dt = 0 due to B<Theta

    mrksize = 10
    if FP[cc][1]=='Unstable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
    if FP[cc][1]=='Semistable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'o',color='white', markersize=mrksize/marker_sizer-3/marker_sizer)
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const,'x',color=col, markersize=mrksize/marker_sizer-3/marker_sizer)
    if FP[cc][1]=='Stable':
        plt.plot(FP[cc][0],FP[cc][0]+k_DR*D_const, 'o',color=col, markersize=mrksize/marker_sizer)

threshold_position = Theta/k_BI
plt.plot([threshold_position, threshold_position],[R_min,R_max],'--',color='tab:red',linewidth = 3.0/marker_sizer)


plt.xlabel("Extent of blood-brain \n" 
           r"barrier disruption $B \approx I$")
plt.ylabel("Degree of circuit \n"
           r"remodeling $R$")
plt.title('$D$='+str(np.round(D_const*100)/100)+r'; $K_{S\rightarrow B}\downarrow$ ',fontsize=font['size'])

ax0.set_xlim([B_min,B_max])
ax0.set_ylim([R_min,R_max])

plt.gcf().subplots_adjust(bottom=margin)
plt.gcf().subplots_adjust(left=margin)
set_size(1.0*w,1.0*w)
fig0.savefig('Figures/Fig8/Fig_8d2.pdf')  

## Figure 8e

In [ ]:
# params=load_default_parameters('rate')

for ii in range (1,len(t_interv_windows[:,0])):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    listt=np.load(DataPath_rate+'/Sim_'+str(ii+1)+'.npy',allow_pickle=True)  
    B_vec=listt[1]['B_vec']
    R_vec=listt[1]['R_vec']
    
    if listt[0]['Complex_input']=='no': # drawing in black color the window of injury presense on the line of simulation results 
        plt.plot(B_vec[0:int(listt[0]['IBDR_E_duration'][1]/listt[0]['dt'])], R_vec[0:int(listt[0]['IBDR_E_duration'][1]/listt[0]['dt'])],color='k',alpha=1,linewidth=1.5)  
    plt.plot(B_vec[0],R_vec[0],'kx',markersize=3,linewidth=0.25)
    plt.plot(B_vec[int(listt[0]['IBDR_E_duration'][1]/listt[0]['dt'])],R_vec[int(listt[0]['IBDR_E_duration'][1]/listt[0]['dt'])],'k*',markersize=3,linewidth=0.25)
    
    # drawing windows of intervention on the line of simulation results
    plt.plot(B_vec[int(t_interv_windows[ii][0]*7/listt[0]['dt']):int(t_interv_windows[ii][1]*7/listt[0]['dt'])],\
             R_vec[int(t_interv_windows[ii][0]*7/listt[0]['dt']):int(t_interv_windows[ii][1]*7/listt[0]['dt'])],\
             color=color_scheme_7examples[ii],linewidth=1,alpha=1)
    plt.plot(B_vec[int(t_interv_windows[ii][0]*7/listt[0]['dt'])],\
             R_vec[int(t_interv_windows[ii][0]*7/listt[0]['dt'])],'+',markersize=4,\
             color=color_scheme_7examples[ii],linewidth=1,alpha=1)
    plt.plot(B_vec[int(t_interv_windows[ii][1]*7/listt[0]['dt'])],\
             R_vec[int(t_interv_windows[ii][1]*7/listt[0]['dt'])],'*',markersize=3,\
             color=color_scheme_7examples[ii],linewidth=1,alpha=1)
    
    #drawing simulation results
    plt.plot(B_vec, R_vec, color=color_scheme_7examples[ii],linewidth=lw_7[ii],alpha=alph_7[ii])
    
    listt=np.load(DataPath_rate+'/Sim_'+str(1)+'.npy',allow_pickle=True) # loading and drawing reference simulaton without intervention
    B_vec=listt[1]['B_vec']
    R_vec=listt[1]['R_vec']
    plt.plot(B_vec, R_vec, color=color_scheme_7examples[0],linewidth=lw_7[0],alpha=alph_7[0])
    
    plt.xlabel("Extent of blood-brain \n" 
               r"barrier disruption $B$")
    plt.ylabel("Degree of circuit \n"
               r"remodeling $R$")

    plt.gcf().subplots_adjust(bottom=margin)
    plt.gcf().subplots_adjust(left=margin)
    plt.gcf().subplots_adjust(top=1-margin/2)
    plt.gcf().subplots_adjust(right=1-margin/2)
    set_size(w,w) 
    fig.savefig('Figures/Fig8/Fig_8e'+str(ii)+'.pdf')